# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect("data.sqlite")

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [7]:
pd.read_sql("""
SELECT customerNumber, contactLastName, contactFirstName
FROM customers
WHERE customerNumber IN (
    SELECT customerNumber
    FROM orders
    WHERE orderDate = "2003-01-31"
)
""", conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [13]:
pd.read_sql("""
SELECT productName,
COUNT(orderNumber) AS num_orders,
SUM(quantityOrdered) AS num_items
FROM products
LEFT JOIN orderdetails
    USING(productCode)
GROUP BY productName
ORDER BY num_items DESC
""", conn)

,productName,num_orders,num_items
0,1992 Ferrari 360 Spider red,53,1808.0
1,1937 Lincoln Berline,28,1111.0
2,American Airlines: MD-11S,28,1085.0
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076.0
4,1930 Buick Marquette Phaeton,28,1074.0
...,...,...,...
105,1911 Ford Town Car,25,832.0
106,1936 Mercedes Benz 500k Roadster,25,824.0
107,1970 Chevy Chevelle SS 454,25,803.0
108,1957 Ford Thunderbird,24,767.0


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [35]:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
pd.read_sql("""
SELECT productName,
COUNT(DISTINCT customerNumber) AS total_people
FROM products
LEFT JOIN orderdetails
    USING(productCode)
LEFT JOIN orders
    USING(orderNumber)
GROUP BY productName
ORDER BY total_people
""", conn).head(10)

,productName,total_people
0,1985 Toyota Supra,0
1,1949 Jaguar XK 120,18
2,1952 Citroen-15CV,18
3,1969 Chevrolet Camaro Z28,18
4,2002 Chevy Corvette,18
5,1958 Chevy Corvette Limited Edition,19
6,1954 Greyhound Scenicruiser,20
7,1957 Ford Thunderbird,20
8,1970 Chevy Chevelle SS 454,20
9,1970 Dodge Coronet,20


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [31]:
pd.read_sql("""
SELECT DISTINCT employeeNumber, firstName, lastName, offices.city, officeCode,
COUNT(DISTINCT customerNumber) AS total_customers
FROM products
LEFT JOIN orderdetails
    USING(productCode)
LEFT JOIN orders
    USING(orderNumber)
LEFT JOIN customers
    USING(customerNumber)
LEFT JOIN employees
    ON customers.salesRepEmployeeNumber = employees.employeeNumber
LEFT JOIN offices
    USING (officeCode)
GROUP BY productName
HAVING total_customers < 20
""", conn)

,employeeNumber,firstName,lastName,city,officeCode,total_customers
0,1501.0,Larry,Bott,London,7.0,18
1,1370.0,Gerard,Hernandez,Paris,4.0,19
2,NaN,None,None,None,NaN,0


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [41]:
pd.read_sql("""
SELECT employeeNumber, firstName, lastName, 
COUNT(customerNumber) AS total_customers
FROM customers
LEFT JOIN employees
    ON customers.salesRepEmployeeNumber = employees.employeeNumber
GROUP BY employeeNumber
HAVING creditLimit > 15000
""", conn)

,employeeNumber,firstName,lastName,total_customers
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!